In [56]:
path2train = "/home/e209440/data/train"
path2test = "/home/e209440/data/test"

In [57]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch as tc
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import transforms, utils

from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm 

In [40]:
import os, glob, math,time
from scipy import misc

In [41]:
img_size = 50
sz = (img_size, img_size)

'''
labels = {'c0' : 'safe driving', 
'c1' : 'texting - right', 
'c2' : 'talking on the phone - right', 
'c3' : 'texting - left', 
'c4' : 'talking on the phone - left', 
'c5' : 'operating the radio', 
'c6' : 'drinking', 
'c7' : 'reaching behind', 
'c8' : 'hair and makeup', 
'c9' : 'talking to passenger'}
'''

labels = {'c0' : 'safe driving'}

In [42]:
def process_image(img_file):
    img = misc.imread(img_file)
    img = misc.imresize(img, sz).transpose((2,0,1)).astype('float32') / 255.0
    return img

In [54]:
def load_traindata(path, labels):
    X_train= []
    Y_train = []
    for j in tqdm(labels):
        start = time.time()
        print('Reading folder {}...'.format(j))
        path_folder = os.path.join(path, str(j), '*.jpg')
        files = glob.glob(path_folder)
        X_train.extend(process_image(im_file) for im_file in files)
        Y_train.extend([j]*len(files))
        end = time.time() - start
        print("Time to load folder "+ j +": %.2f seconds" % end)
    return (X_train,Y_train)

In [55]:
X_train, y_train = load_traindata(path2train, labels)

  0%|          | 0/10 [00:00<?, ?it/s]

c0
Reading folder c0...
Time to load folder c0: 22.62 seconds


 10%|█         | 1/10 [00:22<03:23, 22.63s/it]

c1
Reading folder c1...


KeyboardInterrupt: 

In [8]:
len(X_train)

2489

In [12]:
def load_testdata():
    
    
    start = time.time()
    X_test    = []
    X_test_id = []

    path  = os.path.join(path2test, '*.jpg')
    files = glob.glob(path)

#    results = (process_test_image(im_file) for im_file in files)  # Original

#   Ajuste para pegar poucas imagens : 

    idx = 0
    for im_file in files:
        X_test.extend(process_image(im_file))    
#        results = process_image(im_file) # Lembrar de discutir função process_test_image
            

#    X_test, X_test_id,_ = zip(*results)

    end = time.time() - start
    print("Time: %.2f seconds" % end)
    return (X_test)


In [9]:
X_train[0].shape

(3, 50, 50)

In [10]:
len(y_train)

2489

In [11]:
def process_test_image(img_file):
    return process_image(img_file), os.path.basename(img_file)

In [13]:
X_test = load_testdata()

Time: 1.02 seconds


In [14]:
print(len(X_test))
X_test[2].shape

57


(50, 50)

In [15]:
few_samples = True
if few_samples:
    n_samples = 100
else : 
    n_samples = data['X_train'].shape[0]

In [16]:
X_train = data['X_train'][:n_samples]
y_train = data['y_train'][:n_samples]
X_test = data['X_test'][:n_samples]
y_test = data['y_test'][:n_samples]

NameError: name 'data' is not defined

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
class CRDataset(Dataset):
    
    def __init__(self, data, target, transform):
        assert data.shape[0] == target.shape[0]
        
        self.data = data
        self.target = torch.LongTensor(target.astype(int))
        self.transform = transform
    
    def __len__(self):
        return self.target.shape[0]
    
    def __getitem__(self, i):
        data = self.transform(self.data[i, :, :, :])
        return (data, self.target[i])

